# AI Resume

In [ ]:
from crewai import Agent, Task, Crew, LLM
import litellm
# from litellm import AnthropicChatCompletion

In [ ]:
# Setting up the Serper API Key
from utils import get_serper_api_key, get_gemini_api_key, pretty_print_result

# Retrieve the API keys
serper_api_key = get_serper_api_key()
gemini_api_key = get_gemini_api_key()
file_path = '/resume/sample_resume.docx'


In [ ]:
from langchain_community.llms import Bedrock

In [ ]:
llm = LLM(model="bedrock/anthropic.claude-3-5-haiku-20241022-v1:0", temperature=0.7)

In [ ]:
import crewai_tools

In [ ]:
from crewai_tools import DOCXSearchTool


## Setup CrewAi Tools

In [ ]:
# This code initializes a semantic search tool using Amazon Bedrock as the LLM provider and embedder.
semantic_search_resume = DOCXSearchTool(
    docx=file_path,
    config=dict(
        llm=dict(
            provider="aws_bedrock",  # Using Amazon Bedrock as the LLM provider
            config=dict(
                model="anthropic.claude-3-5-sonnet-20241022-v2:0",  # Specify the Bedrock-supported model
                # temperature=0.5,
                # top_p=1,
                # stream=True,
            ),
        ),
        embedder=dict(
            provider="aws_bedrock",  # Using Amazon Bedrock for embeddings
            config=dict(
                model="amazon.titan-embed-text-v1",  # Bedrock-supported embedding model
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

In [ ]:
# Import necessary tools from crewai_tools module and initialize them
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, FileReadTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path)

In [ ]:
# This code reads a .docx file and displays its content as a Markdown preview in a Jupyter notebook.

from docx import Document

def display_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])  # Extract text from all paragraphs
    return text

file_path = "resume/sample_resume.docx"  # Upload your resume or use a sample 
resume_text = display_docx(file_path)

from IPython.display import display, Markdown
display(Markdown(f"### Resume Preview\n\n{resume_text}"))

## Creating Agents

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [ ]:
# Agent 4: Interview Coach
interview_preparer = Agent(
    role="Engineering Interview Coach",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    llm=llm,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

### Creating Tasks


In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [38]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and linkedin profile "
        "({linkedin_profile}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

In [39]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [40]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

## Creating the Crew

In [41]:
## USE THIS FOR BEDROCK
from langchain_community.llms import Bedrock
 
manager_llm = LLM(model="us.amazon.nova-pro-v1:0", temperature=0.7)

In [42]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],
    llm=manager_llm,
    verbose=True,
    debug=True
)

## Run the Crew

In [43]:
job_application_inputs = {
    'job_posting_url': 'https://www.arionkoder.com/careers/4122129008', # Replace the job posting URL
    'github_url': 'https://github.com/AjBorbzz', # Replace with your GitHub URL
    'linkedin_profile': 'https://www.linkedin.com/in/albert-borbe-489337140/' # Replace with your personal website URL
}

In [44]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

# Agent: Tech Job Researcher
## Task: Analyze the job posting URL provided (https://www.arionkoder.com/careers/4122129008) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
# Agent: Personal Profiler for Engineers
## Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/AjBorbzz) URLs, and linkedin profile (https://www.linkedin.com/in/albert-borbe-489337140/). Utilize tools to extract and synthesize information from these sources.


# Agent: Tech Job Researcher
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.arionkoder.com/careers/4122129008\"}"
## Tool Output: 
Careers at Arionkoder | Join a Team Shaping the Future of Digital Products Services Who We Are Our Work Insights Become an Arionic Reach Arionkoder hello @ arionkoder.com +1(631)204-6094 Services Who We Are Our Work Insights Become an Arionic Contact Us Linke

In [45]:
from IPython.display import Markdown
Markdown(result.raw)

INTERVIEW PREPARATION GUIDE: ALBERT JOHN A. BORBE
Python Developer and Data Engineer Interview Question Bank

TECHNICAL EXPERTISE INTERVIEW QUESTIONS

1. Programming Languages and Frameworks
- Can you walk me through your experience with Python and how you've applied it in real-world projects?
   Talking Points:
   • Highlight advanced Python skills
   • Discuss specific projects from GitHub repositories
   • Emphasize versatility across Django, Flask, and web services

2. Cloud Computing and DevOps
- Describe your experience with AWS and Azure cloud services.
   Talking Points:
   • Reference AWS Solutions Architect certification
   • Explain specific cloud implementation projects
   • Discuss containerization and Docker experience
   • Share your personal philosophy: "Stop trying to learn all of AWS at once"

3. Web Development Technical Questions
- Tell me about your most challenging web development project.
   Talking Points:
   • DjangoReactEcommerce2024 GitHub project
   • Integration of Django and ReactJS
   • Challenges overcome in building web services
   • Responsive design implementation

4. Data Engineering Capabilities
- How do you approach data engineering and machine learning projects?
   Talking Points:
   • Discuss DataScience_AI_ML repository
   • Highlight Coursera certifications in data analysis
   • Explain data pipeline design and optimization techniques
   • Showcase Python data manipulation skills

PROFESSIONAL EXPERIENCE INTERVIEW QUESTIONS

5. Current Role at EY
- Can you describe your responsibilities as a Python Developer and Data Engineer at EY?
   Talking Points:
   • Web application development
   • Cloud solution implementation
   • Data engineering pipeline design
   • Cybersecurity contribution

CERTIFICATION AND CONTINUOUS LEARNING

6. Professional Development
- How do you stay current with emerging technologies?
   Talking Points:
   • Multiple professional certifications
   • Coursera online learning
   • GitHub project contributions
   • Continuous learning mindset

BEHAVIORAL AND SOFT SKILLS QUESTIONS

7. Problem-Solving and Innovation
- Describe a complex technical challenge you've solved recently.
   Talking Points:
   • Provide specific example from EY or personal projects
   • Explain your problem-solving methodology
   • Highlight innovative approach

8. Teamwork and Communication
- How do you collaborate with cross-functional teams?
   Talking Points:
   • Professional communication style
   • Experience working in finance, insurance, banking sectors
   • Ability to explain technical concepts clearly

TECHNICAL SCENARIO QUESTIONS

9. System Design
- How would you design a scalable web application using your current technology stack?
   Talking Points:
   • Discuss microservices architecture
   • Explain cloud deployment strategies
   • Highlight security considerations

10. Future Technology Interests
- Where do you see yourself in the next 3-5 years in tech?
    Talking Points:
    • Continued cloud and data engineering expertise
    • Interest in machine learning and AI
    • Potential certification goals
    • Emerging technology trends

INTERVIEW PREPARATION TIPS:
- Review all certifications in detail
- Prepare concrete examples for each talking point
- Practice explaining technical concepts clearly
- Demonstrate passion for continuous learning
- Highlight your unique personal philosophy and approach to technology

RECOMMENDED PORTFOLIO REVIEW:
- GitHub Repositories
  1. DjangoReactEcommerce2024
  2. DataScience_AI_ML
- Certification Documentation
- Professional Projects at EY

This comprehensive interview preparation guide is designed to help Albert John A. Borbe confidently showcase his technical expertise, professional experience, and continuous learning approach during his interview process.

In [ ]:
#Let's estimate cost. You can see pricing at: https://aws.amazon.com/bedrock/pricing/

import pandas as pd

costs = (0.8 * (job_application_crew.usage_metrics.prompt_tokens) + 3.2*(job_application_crew.usage_metrics.completion_tokens) )/ 1000000
print(f"Total costs: ${costs:.4f}")


# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([job_application_crew.usage_metrics.dict()])
df_usage_metrics